# Urban Heat Islands: India

## Import packages

In [ ]:
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download, plot, utils

plt.style.use("seaborn-v0_8-notebook")

## Define Parameters

In [ ]:
# Region
area = [15, 75, 10, 80]

# Zoom
lon_slice = slice(77.4, 77.8)
lat_slice = slice(12.8, 13.2)

# Periods
satelite_years = {"spot": ["1999", "2000"], "proba": ["2014", "2015"]}

## Define Requests

In [ ]:
collection_id_albedo = "satellite-albedo"
albedo_request = {
    "format": "zip",
    "variable": "albb_bh",
    "sensor": "vgt",
    "product_version": "v2",
    "horizontal_resolution": "1km",
    "month": [f"{month:02d}" for month in range(1, 13)],
    "nominal_day": "10",
    "area": area,
}

collection_id_temperature = "reanalysis-era5-land-monthly-means"
temperature_request = {
    "format": "grib",
    "product_type": "monthly_averaged_reanalysis",
    "variable": "2m_temperature",
    "time": "00:00",
    "month": [f"{month:02d}" for month in range(1, 13)],
    "area": area,
}

## Functions to cache

In [ ]:
def regionalised_func(ds, lon_slice, lat_slice, func):
    ds = utils.regionalise(ds, lon_slice=lon_slice, lat_slice=lat_slice)
    return func(ds)

## Download and transform albedo

In [ ]:
albedo_kwargs = {
    "chunks": {"year": 1, "month": 1},
    "drop_variables": ["crs"],
}

datasets = []
for satellite, years in satelite_years.items():
    ds = download.download_and_transform(
        collection_id_albedo,
        albedo_request | {"satellite": satellite, "year": years},
        transform_func=diagnostics.time_weighted_mean,
        transform_chunks=False,
        **albedo_kwargs,
    )
    datasets.append(ds.expand_dims(satellite=[satellite]))
ds_albedo_maps = xr.concat(datasets, "satellite")

datasets = []
for satellite, years in satelite_years.items():
    ds = download.download_and_transform(
        collection_id_albedo,
        albedo_request | {"satellite": satellite, "year": years},
        transform_func=diagnostics.seasonal_weighted_mean,
        transform_chunks=False,
        **albedo_kwargs,
    )
    datasets.append(ds.expand_dims(satellite=[satellite]))
ds_albedo_season_maps = xr.concat(datasets, "satellite")

albedo_timeseries_zoom = {}
for satellite, years in satelite_years.items():
    albedo_timeseries_zoom[satellite] = download.download_and_transform(
        collection_id_albedo,
        albedo_request | {"satellite": satellite, "year": years},
        transform_func=regionalised_func,
        transform_func_kwargs={
            "lon_slice": lon_slice,
            "lat_slice": lat_slice,
            "func": diagnostics.spatial_weighted_mean,
        },
        **albedo_kwargs,
    )

## Download and transform temperature

In [ ]:
temperature_kwargs = {
    "chunks": {"year": 1},
}

datasets = []
for satellite, years in satelite_years.items():
    ds = download.download_and_transform(
        collection_id_temperature,
        temperature_request | {"year": years},
        transform_func=diagnostics.time_weighted_mean,
        transform_chunks=False,
        **temperature_kwargs,
    )
    datasets.append(ds.expand_dims(satellite=[satellite]))
ds_temperature_maps = xr.concat(datasets, "satellite")

datasets = []
for satellite, years in satelite_years.items():
    ds = download.download_and_transform(
        collection_id_temperature,
        temperature_request | {"year": years},
        transform_func=diagnostics.seasonal_weighted_mean,
        transform_chunks=False,
        **temperature_kwargs,
    )
    datasets.append(ds.expand_dims(satellite=[satellite]))
ds_temperature_season_maps = xr.concat(datasets, "satellite")

temperature_timeseries_zoom = {}
for satellite, years in satelite_years.items():
    temperature_timeseries_zoom[satellite] = download.download_and_transform(
        collection_id_temperature,
        temperature_request | {"year": years},
        transform_func=regionalised_func,
        transform_func_kwargs={
            "lon_slice": lon_slice,
            "lat_slice": lat_slice,
            "func": diagnostics.spatial_weighted_mean,
        },
        **temperature_kwargs,
    )

## Define plotting functions

In [ ]:
def compare_albedo_and_temperature(da_albedo, da_temperature, freq):
    # Resample
    da_albedo = da_albedo.resample(time=freq).mean()
    da_temperature = da_temperature.resample(forecast_reference_time=freq).mean()
    dt = da_temperature["forecast_reference_time"].diff("forecast_reference_time")

    # Convert units
    with xr.set_options(keep_attrs=True):
        da_temperature = da_temperature - 273.15
    da_temperature.attrs["units"] = "°C"

    fig, ax1 = plt.subplots()
    # Plot temperature
    color = "red"
    ax1.bar(
        da_temperature["forecast_reference_time"],
        da_temperature,
        width=dt[0] / 2,
        color=color,
        alpha=0.6,
    )
    ax1.set_ylabel(
        f"{da_temperature.attrs['long_name']} [{da_temperature.attrs['units']}]",
        color=color,
    )
    ax1.tick_params(axis="y", labelcolor=color)

    # Plot albedo
    ax2 = ax1.twinx()
    color = "blue"
    da_albedo.plot(ax=ax2, marker="o", color=color)
    ax2.set_ylabel(ax2.get_ylabel(), color=color)
    ax2.tick_params(axis="y", labelcolor=color)
    ax2.grid()

    ax2.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%b"))
    fig.autofmt_xdate(rotation=45)
    return fig, (ax1, ax2)


def plot_maps(da, lon_slice=None, lat_slice=None, area=area, **kwargs):
    if lon_slice is None:
        lon_slice = slice(*sorted([area[1], area[3]]))
    if lat_slice is None:
        lat_slice = slice(*sorted([area[0], area[2]]))

    kwargs.setdefault("row", "satellite" if "satellite" in da.dims else None)
    kwargs.setdefault("col", "season" if "season" in da.dims else None)
    da["longitude"] = da["longitude"].astype("float32")
    da["latitude"] = da["latitude"].astype("float32")
    da = utils.regionalise(da, lon_slice=lon_slice, lat_slice=lat_slice)

    if da.name == "t2m":
        with xr.set_options(keep_attrs=True):
            da = da - 273.15
        da.attrs["units"] = "°C"

    facet = plot.projected_map(da, **kwargs)
    for ax in facet.axs.flatten():
        ax.set_extent(
            [lon_slice.start, lon_slice.stop, lat_slice.start, lat_slice.stop]
        )
    return facet

## Plot albedo

In [ ]:
custom_cmap_s = mcolors.LinearSegmentedColormap.from_list(
    "CustomColorMap",
    ["blue", "skyblue", "cyan", "lightgreen", "yellow", "orange", "red", "maroon"],
)
facet_albedo = plot_maps(ds_albedo_maps["AL_BH_BB"], cmap=custom_cmap_s)
facet_albedo_zoom = plot_maps(
    ds_albedo_maps["AL_BH_BB"],
    lon_slice=lon_slice,
    lat_slice=lat_slice,
    cmap=custom_cmap_s,
)

## Plot temperature

In [ ]:
facet_albedo = plot_maps(ds_temperature_maps["t2m"], cmap="YlOrRd")
facet_albedo_zoom = plot_maps(
    ds_temperature_maps["t2m"],
    lon_slice=lon_slice,
    lat_slice=lat_slice,
    cmap="YlOrRd",
)

## Plot timeseries

In [ ]:
for freq in ("1MS", "QS-DEC"):
    for satellite, ds_albedo in albedo_timeseries_zoom.items():
        da_albedo = ds_albedo["AL_BH_BB"]
        da_temperature = temperature_timeseries_zoom[satellite]["t2m"]
        fig, (ax1, ax2) = compare_albedo_and_temperature(
            da_albedo, da_temperature, freq
        )
        fig.suptitle(f"{satellite=} {freq=}")
        plt.show()

## Plot seasonal albedo

In [ ]:
facet_albedo = plot_maps(ds_albedo_season_maps["AL_BH_BB"], cmap=custom_cmap_s)
facet_albedo_zoom = plot_maps(
    ds_albedo_season_maps["AL_BH_BB"],
    lon_slice=lon_slice,
    lat_slice=lat_slice,
    cmap=custom_cmap_s,
)

## Plot seasonal temperature

In [ ]:
facet_albedo = plot_maps(ds_temperature_season_maps["t2m"], cmap="YlOrRd")
facet_albedo_zoom = plot_maps(
    ds_temperature_season_maps["t2m"],
    lon_slice=lon_slice,
    lat_slice=lat_slice,
    cmap="YlOrRd",
)